In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Data"

In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import reshape
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU
from keras.utils import to_categorical
from matplotlib import pyplot
import numpy as np
import gc,os,cv2
import asyncio
from time import gmtime, strftime
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [ ]:
def load_dataset5 (fold = 0,prefix=''):
	#print (prefix)


	X1 = read_csv(prefix+'brushing.txt', header=None, delim_whitespace=False) 
	X2 = read_csv(prefix+'rolling.txt', header=None, delim_whitespace=False) 
	X3 = read_csv(prefix+'sliding.txt', header=None, delim_whitespace=False) 
	print (prefix)

	mu = np.average([np.average(X1),np.average(X2),np.average(X3)]) 
	std = np.std([np.std(X1),np.std(X2),np.std(X3)]) 
 
	n_features = 	16
	timelength = 8

	X1=X1.values
	X2=X2.values
	X3=X3.values

	X = np.zeros((X1.shape[0]*3,X1.shape[1]))
	X [ 0:X1.shape[0],:] = X1
	X [X1.shape[0]:X1.shape[0]*2,:]=X2
	X [X1.shape[0]*2:X1.shape[0]*3,:]=X3
	X = (X-mu)/std ; print (" with regularization"); 
	#print ("without regularization")
	X = np.reshape((X),(X.shape[0]//timelength,-1,n_features))


	Y = read_csv(prefix + 'Label.txt', header=None, delim_whitespace=True)
	Y = Y.values
	y = Y
	#y = to_categorical(Y)
	#X,y = shuffle(X,y)
	skf = StratifiedKFold(n_splits=5,shuffle = False,random_state=None)
	#print (skf.get_n_splits(X, y))
	lst = list(skf.split(X,y))
	train_index, test_index  = list(skf.split(X,y))[fold]
	return X[train_index],to_categorical( y[train_index]), X[test_index], to_categorical(y[test_index])

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy,epochs,batch_size):
	#verbose, epochs, batch_size = 1, 56, 100
	#number of parameters of a single LSTM layer: (4 * ((size_of_input + 1) * size_of_output + size_of_output^2))
	verbose = 0
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	#print ("features " , n_features)
	#print ("Outputs " , n_outputs)
	model = Sequential()
# 	model.add(LSTM(100, input_shape=(n_timesteps,n_features),return_sequences=True)) ;print ("first lstm layer 100 neurons") # original 100
	model.add(LSTM(10, input_shape=(n_timesteps,n_features))); print ("LSTM layer 10 neurons") # original 100 ; 
	#model.add(Dropout(0.5))
	#model.add(Dense(10, activation='relu')) # original 200
	model.add(Dense(n_outputs, activation='softmax'))
	#model.summary()
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	history = model.fit(trainX, trainy,  epochs=epochs, batch_size=batch_size, verbose=verbose)
	#history = model.fit(trainX, trainy, validation_split=0.25, epochs=epochs, batch_size=batch_size, verbose=verbose) # it seems worse with validation
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	print ( "verbose ",verbose, " epochs ",epochs, " batch_size ",batch_size)
	return accuracy
    

def evaluate_model_GRU(trainX, trainy, testX, testy,epochs,batch_size):

	verbose = 0
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

	model = Sequential()

	model.add(GRU(10, input_shape=(n_timesteps,n_features))); 
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	history = model.fit(trainX, trainy,  epochs=epochs, batch_size=batch_size, verbose=verbose)
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(myDataset,epochs,batch_size,repeats=3):
	# load data
	trainX, trainy, testX, testy = myDataset
	print('train:', trainX.shape)
	print('train_Label:', trainX.shape)
	print('test:', testX.shape)
	print('test_Label:', testy.shape)
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model_GRU(trainX, trainy, testX, testy,epochs,batch_size)
		#score = evaluate_model(trainX, trainy, testX, testy,epochs,batch_size)
		score = score * 100.0
		#print('>#%d: %.3f' % (r+1, score), strftime("%d/%m/%Y %H:%M:%S +0000", gmtime()))
		scores.append(score)
	# summarize results
	summarize_results(scores)
	print(strftime("%d/%m/%Y %H:%M:%S +0000", gmtime()))

print ("starting ......")
gc.enable()
#for i in [124]:
for  ff in range(0,5):
	myDataset = load_dataset5(fold = ff, prefix = '/content/drive/My Drive/DeepLearning/Average/output=8/')
	print ("starting Fold ", ff )
	for i in [48,96]:
		for j in [48,96]:
		#for j in [96]:
			print ("Fold ", ff, "epochs:",i,", batch: ",j)
			run_experiment(myDataset,epochs=i, batch_size=j,repeats = 10)
			gc.collect()
print ("finished!")
